<a href="https://colab.research.google.com/github/glasgowlab/MAGPIE/blob/GoogleColab/MAGPIE_COLAB.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

<img src="https://github.com/glasgowlab/MAGPIE/blob/GoogleColab/magpie_logo.png?raw=1" alt= “magpie_logo” width="100" align="right">

## **MAGPIE: an interactive tool for visualizing and analyzing many simultaneous protein interactions with a binding partner**

**What is this tool?**

MAGPIE is an online tool hosted on Google Colab that generates 3D visualizations for sets of protein binders interacting with one target ligand (which can be a protein or a small molecule). It also produces sequence logo-style amino acid (AA) frequency graphs. In the AA frequency graphs, the user chooses target ligand positions on the fly, and the size of each binder AA is based on the frequency that it appears in the protein complex dataset within a user-defined distance from the chosen target ligand position(s). The distance is calculated using the alpha carbon positions between protein interfaces, or all heavy atoms in the case of ligands. MAGPIE's 3D visualizer plots the entire target structure, and highlights the residues within a specified distance constraint. There are two options for coloring the interacting residues based on [Amino colour](http://acces.ens-lyon.fr/biotic/rastop/help/colour.htm#aminocolours) and [Shapely colour](http://acces.ens-lyon.fr/biotic/rastop/help/colour.htm#shapelycolours). If you prefer to keep the data private, or have >1000 input PDB files, it is better to clone the [local version](https://github.com/glasgowlab/MAGPIE/tree/local-version).

**Required inputs:**
- PDB files: these should be aligned on the target ligands. Two examples are provided on GitHub, one for protein-protein interactions and one for small molecule-protein interactions.
- Target chain ID from PDB: the program uses the first file in the directory to upload the target ligand structure.
- Protein binder chain ID from PDB: this chain identifier must be the same across all PDB files.
- Target type: indicate whether the target ligand is a small molecule or a protein.
- Target residue index (for protein ligands) or unique atom names (for small molecule ligands): provide the target residue indices for proteins or unique atom names for small molecules. Alternatively, you can input 'all' to consider all AAs/heavy atoms.

**Usage:**
Please execute the cells in numerical order. If you wish to load a different dataset, start from the upload step to reload the new data.


In [ ]:
#@title ##1. Install and import required packages.
%%capture

from google.colab import drive
drive.mount('/content/drive')
!git clone https://github.com/glasgowlab/MAGPIE.git
%cd MAGPIE
!pip install logomaker
!pip install rarfile
!pip install biopython

import pandas as pd
import helper_functions
import sequence_logo_main
import os
import glob
import zipfile
import tarfile
import rarfile
%matplotlib notebook
def configure_plotly_browser_state():
  import IPython
  display(IPython.core.display.HTML('''
        <script src="/static/components/requirejs/require.js"></script>
        <script>
          requirejs.config({
            paths: {
              base: '/static/base',
              plotly: 'https://cdn.plot.ly/plotly-latest.min.js?noext',
            },
          });
        </script>
        '''))
try:
  os.mkdir('temp')
except:
  print("Done")



In [ ]:
#@title ##2. Upload and process data.
# @markdown Navigate to the File menu located top left of the screen. Upload data into the **temp** directory, in the form of a compressed directory in .zip format containing the PDB input files. It is also possible to directly upload the PDB files into the temp folder, but this might take a long time depending on the number of files.
# @markdown
# @markdown   There are two example datasets to test MAGPIE: a small molecule-protein example and a protein-protein example.
# @markdown
# @markdown    Select which dataset you would like to use:
import zipfile


def extract_file(archive_path, output_folder):
    if zipfile.is_zipfile(archive_path):
        with zipfile.ZipFile(archive_path, 'r') as zip_ref:
            zip_ref.extractall(output_folder)
    else:
        print(f"{archive_path} is not a zip file")

uploaded_data_set = False #@param {type:"boolean"}
protein_protein_example = False  #@param {type:"boolean"}
small_molecule_ligand_example = True  #@param {type:"boolean"}
if uploaded_data_set:
  zip_files = glob.glob("temp/" +"*.zip")
  files = zip_files
  pdb_files =[]
  for file in files:
      extract_file(file, "temp")
      file = file.split(".zip")[0]
      pdb_files +=  glob.glob(file + "/*.pdb")

elif small_molecule_ligand_example:
  pdb_files = glob.glob("Small molecule example/reference_1/*.pdb")
elif protein_protein_example:
  pdb_files = glob.glob("Protein Example/" + "*.pdb")

print(pdb_files)

In [ ]:
#@title ##3. Select target ligand and protein binder chains.

# @markdown Select target ligand and protein binder chains, decide whether the target ligand is a ligand or protein, provide heavy atom names (if small molecule ligand) or residue index (if protein ligand) to plot around, and provide the distance in Ångstroms to graph from the target chain.

#@markdown You can also type 'all' to plot all residues or atoms.

# @markdown For the protein target example, use A for the target ligand chain and C for the protein binder chain.

# @markdown For the small molecule target example, use B for the target chain and C for the protein binder chain. Check the is_ligand option.

target_chain = "B"#@param {type:"string"}
binding_chain = "C"#@param {type:"string"}
download_meta = True #@param {type:"boolean"}

is_ligand = True #@param {type:"boolean"}
distance = '8' #@param {type:"string"}
distance = int(distance)

eps = 2
min_samples = 15


# @markdown MAGPIE uses DBSCAN to cluster points in 3d without requiring to specify the number of desired clusters.

# @markdown Run the followng cell if you are interested in changing:

# @markdown eps: The maximum distance between two samples for one to be considered as in the neighborhood of the other.

# @markdown min_samples: The number of samples (or total weight) in a neighborhood for a point to be considered as a core point.


# @markdown Default is eps = 2.0, min_samples = 15

eps = '2' #@param {type:"string"}
min_samples = '15' #@param {type:"string"}

eps = float(eps)
min_samples = int(15)

In [ ]:
#@title #4. Plot points in 3D viewer.
# @markdown This might take a couple of minutes depending on the size of the dataset. It takes around one minute for the protein-protein example.
from matplotlib.pyplot import plot
# configure_plotly_browser_state()
import matplotlib.pyplot as plt

color_key = {
    'Hydrophobic': '#00008B',  # Dark Blue
    'Hydrophilic': '#D02090',  # Pink
    'Aromatic': '#008000',  # Green
    'Charged interactions': '#FF0000',  # Red
}

def plot_cluster_compositions(cluster_compositions_dict):
    plt.ioff()

    # Create a single figure and axis
    fig, ax = plt.subplots(figsize=(12, len(cluster_compositions_dict)*0.9 ))  # Adjusted figsize for larger single plots

    # Define the color for each composition entry
    composition_colors = [color_key['Hydrophobic'], color_key['Hydrophilic'],
                          color_key['Aromatic'], color_key['Charged interactions']]

    # Sort the clusters to maintain order
    sorted_cluster_indexes = sorted(cluster_compositions_dict.keys())
    # Calculate the height of each bar so that there is no space between them
    bar_height = 1.0

    # Loop through each cluster and its composition
    for cluster_index in sorted_cluster_indexes:
        composition = cluster_compositions_dict[cluster_index]
        # The starting point for each bar segment
        left = 0
        # Create each segment of the bar

        for comp, color in  zip(composition, composition_colors):
            ax.barh(cluster_index, comp, height=bar_height, color=color, left=left, edgecolor='white')
            left += comp  # Increment the left position

    # Set the labels and ticks
    ax.set_xlabel('Proportion')
    ax.set_yticks(sorted_cluster_indexes)  # Set y-ticks to cluster indexes
    ax.set_yticklabels([f'Cluster {i}' for i in sorted_cluster_indexes])

    # Remove the spines for a cleaner look
    ax.spines['top'].set_visible(False)
    ax.spines['right'].set_visible(False)
    ax.spines['left'].set_visible(False)
    ax.spines['bottom'].set_visible(False)

    # Create a legend, moved further down by adjusting the bbox_to_anchor parameter
    handles = [plt.Rectangle((0, 0), 1, 1, color=color_key[label]) for label in color_key]
    labels = ["Hydrophobic (G,A,V,C,P,L,I,M)", "Hydrophilic (S,T,N,Q)", "Aromatic (W,F,S)", "Charged (D,E,K,R,H)"]
    plt.legend(handles, labels, loc='upper center', bbox_to_anchor=(0.5, 1.1), ncol=len(color_key))

    filename = "clusters.png"
    plt.savefig(filename)
    plt.ion()  # Turn interactive plotting back on, if needed elsewhere
df_target,df_binder,return_from_clusters = sequence_logo_main.plot(pdb_files, target_chain,binding_chain,is_ligand, distance, download_meta,[eps,min_samples])
graph = False
if len(return_from_clusters) > 0 :
  plot_cluster_compositions(return_from_clusters)
else:
  print("No clusters were found, please change clustering settings or increase dataset size.")


In [ ]:
#@title #4.2 Plot clusters
from IPython.display import Image
import os
import sys
from IPython.display import Image as IPyImage
from PIL import Image as PILImage

def display_image_auto_size(image_path):
    """
    Display an image in a Jupyter notebook.

    Parameters:
        image_path (str): Path to the image file.
    """
    # Open the image file using PIL
    img = PILImage.open(image_path)

    # Get the original width and height of the image
    original_width, original_height = img.size


    # Display the image with the adjusted width and height
    return IPyImage(filename=image_path, width=original_width*.8 , height=original_height*.80)
image_path = 'clusters.png'

if not os.path.isfile(image_path):
  sys.exit("No clusters were found.")

display_image_auto_size(image_path)

In [ ]:


#@title #5. Select target ligand residues or heavy atoms to generate AA frequency graphs.
# @markdown Enter the target residue indices or heavy atom names to graph. These should be separated by commas, without spaces (e.g., N1A,N3A,N9A). Ranges are allowed when working with protein-protein interactions (e.g. 127-131,146-149).
import helper_functions

# @markdown If there are no residues within the given range, the next cell will not execute.
sequence_logo_targets = "N1A,N3A,N9A"#@param {type:"string"}




In [ ]:
#@title #5.  Generate AA frequency graphs.
# @markdown Select if you'd like to only see the combined AA frequency graph.
from IPython.core.pylabtools import figsize
combined = True #@param {type:"boolean" }
plot_list = helper_functions.process_residues_to_graph(sequence_logo_targets,is_ligand)

%matplotlib inline
sequence_logo_main.sequence_logos(df_target,df_binder, plot_list, is_ligand,combined, distance)